In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.util import deprecation
from sklearn.model_selection import train_test_split
deprecation._PRINT_DEPRECATION_WARNINGS = False
# import cv2 as cv
# from tqdm.notebook import tqdm

In [ ]:
def load_images_and_lables(img_path, labels_path):
  '''
  Returns images and labels as np tensor
  '''
  images = np.load(img_path)
  labels = np.load(labels_path)
  return images,labels

print('"load_images_and_lables" function loaded' )

"load_images_and_lables" function loaded


In [ ]:
def preprocess_normalize_images(images, dataset_mean, dataset_std):
  '''
  As suggested in many papers normalization is done using all trainingset mean 
  and std, returns images with std = 1, mean = 0 and of type float32
  '''
  
  # Unfortunatelly CLAHE need to be applied one image at time
  #clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(4,4)) 
  #for idx in tqdm(range(len(images))):
    #images[idx] = clahe.apply(images[idx])

  preprocessed_images = (images - dataset_mean) / dataset_std
  preprocessed_images = preprocessed_images.reshape((preprocessed_images.shape[0], 150, 150, 1))
  preprocessed_images = preprocessed_images.astype('float32')
  return preprocessed_images

print('"preprocess_normalize_images" function loaded' )

"preprocess_normalize_images" function loaded


In [ ]:
def deprocess_image_to_RGB(x):
    '''
    Convert back a preprocessed tensor into a valid image
    '''

    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to grayscale array on 8 bit
    x *= 256
    x = np.clip(x, 0, 256).astype('uint8')
    return x

print('"deprocess_image_to_RGB" function loaded' )

In [ ]:
def deprocess_image(x):
    '''
    Convert back a preprocessed tensor into a valid image
    '''

    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to grayscale array on 16 bit
    x *= 65535
    x = np.clip(x, 0, 65535).astype('uint16')
    return x

print('"deprocess_image" function loaded' )

"deprocess_image" function loaded


In [ ]:
datagen = ImageDataGenerator(
      horizontal_flip = True,
      rotation_range = 30,
      zoom_range = [0.75, 1.25],
      fill_mode = 'nearest')
      #channel_shift_range = 0.2, 
      #vertical_flip = True)


# Parameters from ISMMS
'''
H e V flip
rotation [-25,25]
zoom [0.8, 1.2]
intensity shift [-20, 20] 
'''

# Parameters from Girona & Manchester
'''
H flip
rotation [-30,30]
zoom [0.75, 1.25]
'''

print('"datagen" class loaded' )

"datagen" class loaded


In [ ]:
def binary_subdivision( class0, class1, train_images, train_labels, test_images, test_labels):
  '''
  Extract only the two class needed
  - class0: labels that becomes 0
  - class1: labels that becomes 1
  Returns binarized train and test images and labels
  '''

  binary_train_images = []
  binary_train_labels = []
  binary_test_images = []
  binary_test_labels = []

  for train_image, train_label in zip(train_images, train_labels):
      if train_label in class0:
        binary_train_labels.append(0)
        binary_train_images.append(train_image)
      if train_label in class1:
        binary_train_labels.append(1) 
        binary_train_images.append(train_image)

  for test_image, test_label in zip(test_images, test_labels):
      if test_label in class0:
        binary_test_labels.append(0)
        binary_test_images.append(test_image)
      if test_label in class1:
        binary_test_labels.append(1) 
        binary_test_images.append(test_image)

  binary_train_images = np.array(binary_train_images)
  binary_train_labels = np.array(binary_train_labels)
  binary_test_images = np.array(binary_test_images)
  binary_test_labels = np.array(binary_test_labels)

  return binary_train_images, binary_train_labels, binary_test_images, binary_test_labels

print('"binary_subdivision" function loaded' )

"binary_subdivision" function loaded


In [ ]:
def split_train_validation(train_images, train_labels, validation_split):
  '''
  Split dataset in validation and training set and returns them (OBSOLETE)
  '''
  # Shuffle the data
  seed = 711997 
  rng = np.random.RandomState(seed)
  rng.shuffle(train_images)
  rng = np.random.RandomState(seed)
  rng.shuffle(train_labels)

  # Extract a training & validation split
  num_validation_samples = int(validation_split * len(train_labels))
  split_val_images = train_images[-num_validation_samples:]
  split_train_images = train_images[:-num_validation_samples]
  split_val_labels = train_labels[-num_validation_samples:]
  split_train_labels = train_labels[:-num_validation_samples]
  return split_train_images, split_train_labels, split_val_images, split_val_labels

  print('"split_train_validation" function loaded' )

In [ ]:
def split_train_validation_baseline(train_images, train_labels, validation_split, baselines):
  '''
  Split dataset in validation and training set and returns them
  '''

  train_indexes, val_indexes, split_train_labels, split_val_labels = train_test_split(range(train_images.shape[0]), train_labels, test_size=0.2, random_state=171997, stratify=train_labels)
  
  # Extract a training & validation split
  split_val_images = train_images[val_indexes]
  split_train_images = train_images[train_indexes]
  val_baselines = baselines[val_indexes]
  train_baselines = baselines[train_indexes]

  return split_train_images, split_train_labels, split_val_images, split_val_labels, val_baselines, train_baselines

  print('"split_train_validation_baseline" function loaded' )